In [2]:
import time 
import rtmidi
import re

In [8]:
def MidiIn(n,q1,q5):    
    
    #Setup midi port
    midi_in = rtmidi.MidiIn()
    ports = midi_in.get_ports() 
    ports_dict = {k: v for (v,k) in enumerate(ports)}
    name_Keyword = "Arturia BeatStep Pro" #Change this to the name or part of the name of your midi device e.g "Arturia" or "Beatstep" would also work here.
    #Was having issues with the port number being attached to the name
    
    for k  in ports_dict:
        if name_Keyword in k:         
            port_name = k
            print(port_name)
            break
        
    midi_in.open_port(ports_dict[k])
    #midi_in.open_port(2)
    
    record = False;      # Set record catch off
    hold = False;       # Set hold note catch off
    midi_chan = 0;      # 0-15 -> 1-16
    output = []  # Init output with some size to remove later
    dur = [0]*127
    sleep_st = time.time()
    sleep_t = 0
    t_end = time.time()*2
    count = 0
    
    nNotes = 0         
    
    while midi_in.is_port_open() and sleep_t <10 :
        while sleep_t < 10 and nNotes < n+1:            
            msg_and_dt = midi_in.get_message()
            #Process the message        ([cmd, pitch, vel], dt)
            if msg_and_dt:
                sleep_st = time.time()
                record = True
                
                (msg, dt) = msg_and_dt      #Split up message and delay time
                command = hex(msg[0])   #Convert into hex
                
                if command == hex(int(0x90)+midi_chan) and not hold: #0x90 note on midi chan 1
                    dur[msg[1]] = 0
                    dur_last = 0
                    
                    if output:
                        dur_last = output[len(output)-1][2]
                        if type(dur_last)==list:
                            print('message breaking1',msg[0])
                            break
                    hold = True #Set hold note catch on
                    output.append([msg[1],dur_last+dt,dur])#((msg[1],dur_last+dt,dur))

                    
                elif command == hex(int(0x80)+midi_chan) and hold and output: #force monophonic keyboard with hold bool
                    if msg[1] == output[len(output)-1][0]:
                        lst_t = list(output[len(output)-1])    #list with [pitch, step, dur]
                        lst_t[2] = dur[msg[1]]+dt              #update the duration
                        #print('duration',lst_t[2],dur[msg[1]]) # Note duration check                                                       
                        
                        #Add individual note data to queue
                        q1.put(lst_t) 
                        print(lst_t)
                        output[len(output)-1] = tuple(lst_t) 
                        
                        #Reset variable
                        dur[msg[1]] = 0
                        hold = False
                        nNotes += 1
                        print(nNotes)
                    
                else: #Ignore other midi messages but keep the duration stored
                    dur[msg[1]]= dur[msg[1]]+dt
                    if not (type(dur[msg[1]])== int or type(dur[msg[1]])== float):
                            print('message breaking2',msg[0])
                            break
                    
                    
                    
                    
            
            else:                
                if record == False:
                    t_end = time.time() + 5
                    sleep_t = 0
                
                sleep_t = time.time() - sleep_st
                time.sleep(0.001)
        else:#if sleep_t < 10:
            if sleep_t <= 10 and not hold:
                #Output 5 notes list of tuples
                q5.put(output)               
                print(output) #send output here
                #Reset variables
                nNotes = 0
                output = []
                record = False
                t_end = time.time() + 5
            elif hold:
            #if a note is being held - either extend time or force the output
                t_end = time.time() + 0.1   #extend time to wait for note off
                print('time extended-held note:',msg[1],sum(dur),'seconds')
                
                count += 1
                #if count ==30:
                    #break
                
            elif sleep_t > 10:
                print('sleepy time')
                break
    print('thanks for playing')
    midi_in.close_port()
    time.sleep(1)

blah
Arturia BeatStep Pro 0
[62, 0.0, 0.3620000000000001]
1
[62, 0.5870000000000001, 0.34400000000000014]
2
[62, 0.6400000000000001, 0.38600000000000007]
3
[62, 0.675, 0.4450000000000002]
4
[62, 0.7690000000000001, 0.4270000000000001]
5
[62, 0.929, 0.2790000000000001]
6
[(62, 0.0, 0.3620000000000001), (62, 0.5870000000000001, 0.34400000000000014), (62, 0.6400000000000001, 0.38600000000000007), (62, 0.675, 0.4450000000000002), (62, 0.7690000000000001, 0.4270000000000001), (62, 0.929, 0.2790000000000001)]
[62, 0.631, 0.2790000000000001]
1
[62, 0.8270000000000002, 0.3340000000000001]
2
[62, 0.7630000000000001, 0.31500000000000006]
3
[62, 0.653, 0.2780000000000001]
4
[62, 0.5670000000000001, 0.25600000000000006]
5
[62, 0.5330000000000001, 0.26700000000000007]
6
[(62, 0.631, 0.2790000000000001), (62, 0.8270000000000002, 0.3340000000000001), (62, 0.7630000000000001, 0.31500000000000006), (62, 0.653, 0.2780000000000001), (62, 0.5670000000000001, 0.25600000000000006), (62, 0.5330000000000001, 

KeyboardInterrupt: 

In [ ]:
#Check midi connections open and close properly
midi_in = rtmidi.MidiIn()
ports = midi_in.get_ports()
print(ports)

ports_dict = {k: v for (v,k) in enumerate(ports)}
name_Keyword = "Arturia BeatStep Pro" #Change this to the name or part of the name of your midi device e.g "Arturia" or "Beatstep" would also work here.
#Was having issues with the port number being attached to the name

for k  in ports_dict:
    if name_Keyword in k:         
        port_name = k
        print(port_name)
        break
    
midi_in.open_port(ports_dict[k])
time.sleep(10)
midi_in.close_port()
print(midi_in.is_port_open())
time.sleep(1)
print('done')

In [7]:
#Check midi ports
midi_in = rtmidi.MidiIn()
ports = midi_in.get_ports()
print(ports)

['Arturia BeatStep Pro 0', 'MIDIIN2 (Arturia BeatStep Pro) 1']
